In [2]:
#import scrublet as scr
import numpy as np
import pandas as pd
import statistics as st
import re
import csv
import scanpy as sc 
import scanpy.external as sce
import phate
import matplotlib
from matplotlib import pyplot as plt
import warnings
import pickle
from scipy.stats import binom
from scipy.stats import multinomial
import seaborn as sns
from scipy.stats import hypergeom
import warnings
warnings.filterwarnings('ignore')
import scrublet as scr
import scipy
import itertools
import harmonypy as hm
from random import sample
import matplotlib.patches as mpatches
from scipy.stats import linregress



from sgRNA_Filtering_Functions import CB_Filter_SG_Stat
from sgRNA_Filtering_Functions import CB_Filter_SG_Stat_ALL

# Load Data

In [131]:
#SG_DF = pd.read_pickle('/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/PICKLES/DAP3_SGRNA_IND_072821.pkl')

In [132]:
#CM_Cells = sc.read_h5ad('/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/H5AD/DAP3_SC_CM_Pseudo_PHATE_Man_080521.h5ad')

In [133]:
Single_Guide_List= ['NC_GAG', 'NC_GFP', 'NC_GFP_1', 'NC_GFP_2', 'NC_HS2', 'MALAT1']

In [134]:
PATH = '/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/DIC/'
Dict_File = open(PATH + "Guide_Dictionary.pkl", "rb")
Guide_Dictionary = pickle.load(Dict_File)

### CM Filtered

In [15]:
PATH = '/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/SG_FILTERING/'

Filtered_Targets = []
Filtered_SG = []
for Target in Guide_Dictionary:
    if Target in Single_Guide_List:
        continue
    
    Enr_Guides, Dep_Guides, Faulty_Guides = CB_Filter_SG_Stat(pval_path=PATH, target=Target, df_suffix='_080521_', dict_suffix='_combo_dict_080521', total_sg=396)
    #Faulty_Guides = CB_Filter_SG_Stat(pval_path=PATH, target=Target, df_suffix='_072621_', dict_suffix='_combo_dict_072621')
    if len(Faulty_Guides) > 0:
        Filtered_Targets.append(Target)
        Filtered_SG.append(Faulty_Guides)

FILTERED_DICT = dict(zip(Filtered_Targets,Filtered_SG))

In [17]:
a_file = open(PATH + "CM_FILTERED_DICT_080521.pkl", "wb")
pickle.dump(FILTERED_DICT, a_file)
a_file.close()

### ALL Filtered

In [18]:
PATH = '/project/GCRB/Hon_lab/s425140/03.Data/07.Cardiomyocyte_Small_Scale/DAP3/07_28_21_Paper/SG_FILTERING/'

Filtered_Targets = []
Filtered_SG = []
for Target in Guide_Dictionary:
    if Target in Single_Guide_List:
        continue
    
    Enr_Guides, Dep_Guides, Faulty_Guides = CB_Filter_SG_Stat_ALL(pval_path=PATH, target=Target, df_suffix='_080321_', dict_suffix='_combo_dict_080321_ALL', total_sg=396)
    if len(Faulty_Guides) > 0:
        Filtered_Targets.append(Target)
        Filtered_SG.append(Faulty_Guides)

FILTERED_DICT_ALL = dict(zip(Filtered_Targets,Filtered_SG))

In [19]:
a_file = open(PATH + "ALL_FILTERED_DICT_080521.pkl", "wb")
pickle.dump(FILTERED_DICT_ALL, a_file)
a_file.close()

### Replace sgRNA cells based on CM Filtered

In [135]:
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "CM_Large_sgRNA_Dic.pkl", "rb")
CM_Cells_Dic = pickle.load(Dict_File)

In [136]:
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "ALL_Large_sgRNA_Dic.pkl", "rb")
ALL_Cells_Dic = pickle.load(Dict_File)

In [137]:
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "Filtered_CM_NC.pkl", "rb")
Filtered_NC_CM = pickle.load(Dict_File)

In [3]:
PATH = '../../Data/Pickles/'
Dict_File = open(PATH + "CM_FILTERED_DICT_080521.pkl", "rb")
CM_FILTERED_DICT = pickle.load(Dict_File)

In [139]:
Targets = []
Target_SG = []

for target in Guide_Dictionary.keys():
    
    filtered_guides = []
    
    if target not in CM_FILTERED_DICT.keys():
        continue
    
    faulty_guides = []
    for sg_filt in CM_FILTERED_DICT[target]:
        faulty_guides.append(Guide_Dictionary[target][sg_filt - 1])
        
    for sg in Guide_Dictionary[target]:
        if sg not in faulty_guides:
            filtered_guides.append(sg)
            
    Guide_Dictionary[target] = filtered_guides   

In [141]:
Filtered_NC_List = [i for i in Guide_Dictionary['NC'] if i not in Filtered_NC_CM]
Guide_Dictionary['NC'] = Filtered_NC_List

In [142]:
del CM_Cells_Dic[Filtered_NC_CM]
del ALL_Cells_Dic[Filtered_NC_CM]

In [143]:
PATH = '../../Data/Pickles/'

In [144]:
### Save Guide Dictionary
a_file = open(PATH + "Guide_Dictionary_Filtered.pkl", "wb")
pickle.dump(Guide_Dictionary, a_file)
a_file.close()

In [145]:
### Save Guide Dictionary
a_file = open(PATH + "CM_Large_sgRNA_Dic_Filtered.pkl", "wb")
pickle.dump(CM_Cells_Dic, a_file)
a_file.close()

In [146]:
### Save Guide Dictionary
a_file = open(PATH + "ALL_Large_sgRNA_Dic_Filtered.pkl", "wb")
pickle.dump(ALL_Cells_Dic, a_file)
a_file.close()